# Signal Of Interest Pipeline Demonstration
This notebook is designed to utilize the source code of this project to generate multiple OFDM signals, filter the signal of interest in the contested band, and then generate spectrograms of the result.

## Setup
The following code is required for an instance of Google Colab to have the correct dependencies. 

In [ ]:
import sys

import os 

if "google.colab" in sys.modules:
    !pip -q install -r https://raw.githubusercontent.com/bouchard-josiah/dsp_pipeline/main/requirements.txt

Now, it's time to import the necessary libraries and source code. 

After this, the number of signals in our band, our collection bandwidth, and the time and frequency location and duration of each OFDM signal need to be configured. 

In [ ]:
# Dependencies

import numpy as np

import string

from pathlib import Path

from dsp_demo.OFDM_Gen import OFDM, configs

from dsp_demo.FIR_Filter import Filters


# Configs

spec_bins_t = 1000

noise_pwr = 1

FFT_taps = 4096

samp_rate = 100e6

center_freqs = np.array([25e6, 0, -30e6])

msg_carriers = np.array([300,400,500])

msg_symbols = np.array([400,200,300])

msg_start = np.array([100, 300, 400]) #bins offset

rx_sig = np.zeros([spec_bins_t*FFT_taps], dtype = complex)

sig_num = center_freqs.size



## Signal Generation
Here, the OFDM class will be used to generate the configured signals.

In [ ]:
    cfg = configs(sample_rate = samp_rate, FFT_taps = FFT_taps, spec_len = spec_bins_t, noise_pwr = noise_pwr)

    for i in range(0,sig_num):

        OFDM_loc = OFDM(cfg, msg_carriers = msg_carriers[i], tx_symbs = msg_symbols[i], t_offset_bins = msg_start[i], f_offset = center_freqs[i])

        OFDM_loc.msg_gen()

        OFDM_loc.IFFT_sig_BPSK_mod(cfg)

        rx_sig += OFDM_loc.signal_t

    OFDM_loc.add_noise(cfg, signal = rx_sig)

    OFDM_loc.gen_spectro(cfg)